Let's take a look at some card-level data, courtesy of mtgjson.
Here's specifically where to find what I'm using:
https://mtgjson.com/downloads/all-files/

In [148]:
# data_path = '/Users/connorkenehan/Documents/GitHub/gathering-data/data/AtomicCards.json.zip'
data_path = '/Users/connorkenehan/Downloads/AtomicCards.json'
import pandas as pd
data = pd.read_json(data_path)

In [149]:
#let's see how the whole dataframe looks
data.head()

,meta,data
date,2023-08-24,NaN
version,5.2.1+20230824,NaN
"""Ach! Hans, Run!""",NaN,"[{'colorIdentity': ['G', 'R'], 'colors': ['G',..."
"""Brims"" Barone, Midway Mobster",NaN,"[{'colorIdentity': ['B', 'W'], 'colors': ['B',..."
"""Lifetime"" Pass Holder",NaN,"[{'colorIdentity': ['B'], 'colors': ['B'], 'co..."


In [150]:
#that third field looks funky, let's check out an individual row
data.iloc[2]['data']

[{'colorIdentity': ['G', 'R'],
  'colors': ['G', 'R'],
  'convertedManaCost': 6.0,
  'firstPrinting': 'UNH',
  'foreignData': [],
  'identifiers': {'scryfallOracleId': 'a2c5ee76-6084-413c-bb70-45490d818374'},
  'isFunny': True,
  'layout': 'normal',
  'legalities': {},
  'manaCost': '{2}{R}{R}{G}{G}',
  'manaValue': 6.0,
  'name': '"Ach! Hans, Run!"',
  'printings': ['UNH'],
  'purchaseUrls': {'cardKingdom': 'https://mtgjson.com/links/84dfefe718a51cf8',
   'cardKingdomFoil': 'https://mtgjson.com/links/d8c9f3fc1e93c89c',
   'cardmarket': 'https://mtgjson.com/links/b9d69f0d1a9fb80c',
   'tcgplayer': 'https://mtgjson.com/links/c51d2b13ff76f1f0'},
  'subtypes': [],
  'supertypes': [],
  'text': 'At the beginning of your upkeep, you may say "Ach! Hans, run! It\'s the . . ." and the name of a creature card. If you do, search your library for a card with that name, put it onto the battlefield, then shuffle. That creature gains haste. Exile it at the beginning of the next end step.',
  'type':

When we take a quick look at how our dataframe appears from the get-go, it's
clear that we need to do some additional work to transform it into something
we can easily work with and learn from.  Let's clean that data!

Some observations from eyeballing the first few rows:
1. We have three fields.
2. The first field seems to be a card name
3. The second field, called meta, appears to be metadata which we won't actually need while analyzing cards
4. The third field, called data, seems to have everything we need in it...it's just all smooshed in a dictionary inside of a list.  So we'll have to unpack that a bit, but it's the only field we need out of this entire json!

Let's get into it and clean that data!

We know that there's some amount of null values in this dataset.  How much, and in what fields?

In [284]:
cleaned_data = data.reset_index()

def count_nulls(data, column):
    na_count = data[column].isna().sum()
    print('\n',column,':',na_count,'null rows')

print(len(cleaned_data),'total rows')
for column in cleaned_data.columns:
    count_nulls(cleaned_data, column)


27352 total rows

 index : 0 null rows

 meta : 27350 null rows

 data : 2 null rows


Ok, so these counts tell us that our meta column is only those two rows we saw when we looked at the top chunk of the dataset.  We also learn that our main (data) field only has two nulls in it, which we also saw earlier.  Let's drop the meta field and those null rows.

Finally, since we know that our data field contains everything we need, let's drop the index field and transform the contents of data into individual data fields we can do analysis on more easily.

In [285]:
keep_list = ['data']
keep_data = cleaned_data[keep_list].dropna()
keep_data.head()

,data
2,"[{'colorIdentity': ['G', 'R'], 'colors': ['G',..."
3,"[{'colorIdentity': ['B', 'W'], 'colors': ['B',..."
4,"[{'colorIdentity': ['B'], 'colors': ['B'], 'co..."
5,"[{'colorIdentity': ['B'], 'colors': ['B'], 'co..."
6,"[{'colorIdentity': ['W'], 'colors': ['W'], 'co..."


In [291]:
#assign a lambda function which 
#1. casts each dataframe row as a list
#2. pulls the nth element out of each list value

#then transform the resultant dictionary from key-values into columns

dict_data = keep_data.assign(dict_data = lambda x: (x['data'].str[0]))[['dict_data']]
dict_data = pd.json_normalize(dict_data['dict_data'])
dict_data.head()


,colorIdentity,colors,convertedManaCost,firstPrinting,foreignData,isFunny,layout,manaCost,manaValue,name,...,isReserved,legalities.oldschool,purchaseUrls.cardKingdomEtched,purchaseUrls.tcgplayerEtched,hand,life,colorIndicator,asciiName,hasAlternativeDeckLimit,defense
0,"[G, R]","[G, R]",6.0,UNH,[],True,normal,{2}{R}{R}{G}{G},6.0,"""Ach! Hans, Run!""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[B, W]","[B, W]",5.0,UNF,[],True,normal,{3}{W}{B},5.0,"""Brims"" Barone, Midway Mobster",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[B],[B],1.0,UNF,[],NaN,normal,{B},1.0,"""Lifetime"" Pass Holder",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[B],[B],3.0,UST,[],True,normal,{2}{B},3.0,"""Rumors of My Death . . .""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[W],[W],2.0,AFR,[],NaN,normal,{1}{W},2.0,+2 Mace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
